In [ ]:
%load_ext nb_black

In [ ]:
import cv2

import uuid

import os

import time

In [ ]:
img_labels = ["sad", "happy", "angry"]
num_img = 9

In [ ]:
Img_path = os.path.join(".", "Data", "Images")

In [ ]:
MODEL_name = "ssd_mobnet_emotion"
Pretrained_model_name = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
Pretrained_model_url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"

In [ ]:
file_paths = {
    "Images_path": os.path.join(".", "Data", "Images"),
    "Labelled_images_path": os.path.join(".", "Data", "Labelled Images"),
    "Train_images_path": os.path.join(".", "Data", "Train_images"),
    "Test_images_path": os.path.join(".", "Data", "Test_images"),
    "Protoc_path": os.path.join(".", "Models", "Protoc"),
    "Models_path": os.path.join(".", "Models"),
    "Pretrained_model_path": os.path.join(".", "Models", "pretrained_model"),
    "Output_path": os.path.join(".", "Models", MODEL_name, "Export"),
    "Miscellaneous_path" : os.path.join('.', 'Miscellameous')
}

In [ ]:
if not os.path.exists(Img_path):
    if os.name == "nt":
        !mkdir {Img_path}
    if os.name == "posix":
        !mkdir -p {Img_path}
for label in img_labels:
    label_path = os.path.join(Img_path, label)
    if not os.path.exists(label_path):
        if os.name == "nt":
            !mkdir {label_path}
        if os.name == "posix":
            !mkdir -p {label_path}

In [ ]:
def image_capture(img_labels, Img_path, num_img):
    for img_label in img_labels:
        capture_device = cv2.VideoCapture(0)
        print(f"Collecting images for {img_label}")
        time.sleep(6)
        for img in range(num_img):
            print(f"Collecting image no.{img+1}")
            retina, frame = capture_device.read()
            img_name = os.path.join(
                Img_path, img_label + "." + f"{str(uuid.uuid1())}.jpg"
            )
            cv2.imwrite(img_name, frame)
            cv2.imshow("frame", frame)
            time.sleep(3)

            ## Manual Exit Sequence For the capture using the 'q' key
            if (cv2.waitKey(1) & 0xFF) == ord("q"):
                break

    capture_device.release()
    cv2.destroyALLWindows()

In [ ]:
# !pip install PyQt5

In [ ]:
# !pip install labelImg

In [ ]:
def image_labelling():
    LabelImg_path = os.path.join(".", "Data", "Labelled Images")

    if not os.path.exists(LabelImg_path):
        if os.name == "nt":
            !mkdir {LabelImg_path}
        if os.name == "posix":
            !mkdir -p {LabelImg_path}

    ## Installation setup
    if os.name == "posix":
        !cd {LabelImg_path} && make qt5py3
    elif os.name == "nt":
        !cd {LabelImg_path} && pyrcc5 -o libs/resources.py resources.qrc

    ## Opening the labelImg app
    !cd {LabelImg_path} && python labelImg.py

In [ ]:
### SPLIT THE IMAGES INTO TRAIN AND TEST DATA FOLDERS

In [ ]:
for path in file_paths.values():
    if not os.path.exists(path):
        if os.name == "nt":
            !mkdir {path}
        if os.name == "posix":
            !mkdir -p {path}

In [ ]:
# ! pip install wget

In [ ]:
import wget

In [ ]:
## Downloading Tensor Flow Object Detection API
if not os.path.exists(os.path.join(file_paths["Models_path"], "object-detection-api")):
    ! mkdir {os.path.join(file_paths['Models_path'], 'object-detection-api')}
    !git clone https://github.com/tensorflow/models {os.path.join(file_paths['Models_path'], 'object-detection-api')}

In [ ]:
## Installing Object Detection

if os.name == "posix":
    !apt-get install protobuff-compiler
    !cd Models/object-detection-api/research && protoc object_detection/protos/*.proto --python_out. && cp object_detection/packages/tf2/setup.py

if os.name == "nt":
    url = "https://github.com/protocolbuffers/protobuf/releases/download/v22.0/protoc-22.0-win64.zip"
    wget.download(url)
    !move protoc-22.0-win64.zip {file_paths['Protoc_path']}
    ! cd {file_paths['Protoc_path']} && tar -xf protoc-22.0-win64.zip
    os.environ["PATH"] += os.pathsep + os.path.abspath(
        os.path.join(file_paths["Protoc_path"], "bin")
    )
    !cd Models/object-detection-api/research && protoc object_detection/protos/*.proto -- python_out=. && copy object_detection/packages/tf2/setup.py
    ! cd Models/object-detection-api/research/slim && pip install -e .

In [ ]:
## Object Detection Setup Verification
Verification_script = os.path.join(
    file_paths["Models_path"],
    "object-detection-api",
    "research",
    "object_detection",
    "builders",
    "model_builder_tf2_test.py",
)
!python {Verification_script}

In [ ]:
!pip install tensorflow --upgrade

In [ ]:
!pip uninstall protobuf matplotlib -y
!pip install protobuf matplotlib

In [ ]:
import object_detection

In [ ]:
if os.name == 'posix':
    !wget {Pretrained_model_url}
    !mv {Pretrained_model_name + '.tar.gz'} {file_paths['Pretrained_model_path']}
    !cd {file_paths['Pretrained_model_path']} && tar -zxvf {Pretrained_model_name + '.tar.gz'}

if os.name == 'nt':
    wget.download(Pretrained_model_url)
    !move {Pretrained_model_name + '.tar.gz'} {file_paths['Pretrained_model_path']}
    !cd {file_paths['Pretrained_model_path']} && tar -zxvf {Pretrained_model_path + '.tar.gz'}

In [ ]:
labels = [{'name' : 'happy', 'id' : 0},{'name' : 'sad', 'id' : 1},{'name' : 'angry', 'id' : 2}] 
with open(file_paths['Miscellaneous_path'], 'w') as file:
    for label in labels:
        file.write('item { \n')
        file.write(f'\tname:\{label['name']}\n')
        file.write(f'\tid:{label['id']}\n')
        file.write('}\n')

In [ ]:
ARCHIVE_FILES = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')
if os.path.exists(ARCHIVE_FILES):
    !tar -zxvf {ARCHIVE_FILES}
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')}

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}


In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
def config:
    config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
    pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
    with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
        proto_str = f.read()                                                                                                                                                                                                                                          
        text_format.Merge(proto_str, pipeline_config)  
    
    pipeline_config.model.ssd.num_classes = len(labels)
    pipeline_config.train_config.batch_size = 4
    pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
    pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
    pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
    pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
    pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
    pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

    config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
    with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
        f.write(config_text)   

In [ ]:
## Training
def training_():
    TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
    ! python {TRAINING_SCRIPT} --model_dir={paths['CHECKPOINT_PATH']} --pipeline_config_path={files['PIPELINE_CONFIG']} --num_train_steps=2000

In [ ]:
## Testing
def testing_():
    ! python {TRAINING_SCRIPT} --model_dir={paths['CHECKPOINT_PATH']} --pipeline_config_path={files['PIPELINE_CONFIG']} --checkpoint_dir={paths['CHECKPOINT_PATH']}

In [ ]:
training_()
testing_()

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-5')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections